In [1]:
import os

from dataclasses import dataclass

import numpy as np
import pandas as pd

import andes
import ams

import cosim

In [2]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [3]:
case_path = "./../cases"
res_path = "./../results"
addfile = case_path + '/IL200_dyn_db2.xlsx'

# --- file loading ---
curve = pd.read_csv(case_path + '/CurveInterp.csv')

In [4]:
sp = ams.load(case_path + '/IL200_rted.xlsx',
              setup=True, no_output=True,
              default_config=True)
sa = sp.to_andes(addfile=addfile,
                 setup=True, no_output=True,
                 default_config=True)

Generating code for 1 models on 12 processes.


In [ ]:
# NOTE: 1) the maximum number of dispatch should follow: "D * Dispatch_interval <= 3600"
#       2) the maximum number of AGC should follow: "Dispatch_interval % AGC_interval == 0"

Dispatch_interval = 600  # seconds
AGC_interval = 15  # seconds

for hour in [22]:  # 0, 8, 12, 18
    for dispatch in [9]:
        print(f'--- Running hour {hour}, dispatch {dispatch} ---')
        r0 = hour * 3600 + dispatch * Dispatch_interval
        r1 = r0 + Dispatch_interval
        curve_dispatch = curve.iloc[r0:r1].reset_index(drop=True)

        res_csv = os.path.join(res_path, f'case1_h{hour}_d{dispatch}.csv')

        sim = cosim.CoSimulation(sp=sp, sa=sa,
                                 curve=curve_dispatch,
                                 Dispatch_interval=Dispatch_interval,
                                 AGC_interval=AGC_interval,
                                 addfile=addfile,
                                 res_csv=res_csv,)

        sim.run()

--- Running hour 22, dispatch 9 ---
Load: 940.04 MW, Gen.: 940.04 MW Cost: 30814.18 $


/Users/jinningwang/work/demo/demo/deadband/notes/cosim.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.link = self.sp.dyn.link.copy().fillna(False)


Watchdog: Second: 100
Watchdog: Second: 200
Watchdog: Second: 300
Watchdog: Second: 400
Watchdog: Second: 500
Co-simulation completed. Results saved to ./../results/case1_h22_d9.csv
